<a href="https://colab.research.google.com/github/TalhaOmi/unet_finger_vein_tensor_federated/blob/main/unet_with_tensor(federated).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import imageio
from albumentations import HorizontalFlip, VerticalFlip, ElasticTransform, GridDistortion, OpticalDistortion, CoarseDropout

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path):
    """ X = Images and Y = masks """

    train_x = sorted(glob(os.path.join(path, "train", "set1", "image", "*")))
    train_y = sorted(glob(os.path.join(path, "train", "set1", "mask", "*")))

    test_x = sorted(glob(os.path.join(path, "valid", "set1", "image", "*")))
    test_y = sorted(glob(os.path.join(path, "valid", "set1", "mask", "*")))

    return (train_x, train_y), (test_x, test_y)


if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)

    """ Load the data """
    data_path = "/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/"
    (train_x, train_y), (test_x, test_y) = load_data(data_path)

    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Test: {len(test_x)} - {len(test_y)}")


Train: 19 - 19
Test: 5 - 5


# Model

In [3]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet_main(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="UNET")
    return model

if __name__ == "__main__":
  input_shape = (512, 512, 3)
  model_main = build_unet_main(input_shape)
  model_main.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 512, 512, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 512, 512, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                              

# Metrics

In [4]:

import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

# Train

In [5]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision


H = 512
W = 512

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path):
    x = sorted(glob(os.path.join(path, "set1","image", "*")))
    y = sorted(glob(os.path.join(path, "set1","mask", "*")))
    return x, y

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)              ## (512, 512, 1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch_size=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(4)
    return dataset

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory to save files """
    create_dir("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files")

    """ Hyperparameters """
    batch_size = 1
    lr = 1e-4
    num_epochs = 20
    model_path = os.path.join("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files", "model.h5")
    csv_path = os.path.join("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files", "data.csv")

    """ Dataset """
    dataset_path = "/content/drive/MyDrive/Colab Notebooks/Dataset/new_data"
    train_path = os.path.join(dataset_path, "train")
    valid_path = os.path.join(dataset_path, "valid")

    train_x, train_y = load_data(train_path)
    train_x, train_y = shuffling(train_x, train_y)
    valid_x, valid_y = load_data(valid_path)

    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Valid: {len(valid_x)} - {len(valid_y)}")

    train_dataset = tf_dataset(train_x, train_y, batch_size=batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch_size=batch_size)

    train_steps = len(train_x)//batch_size
    valid_setps = len(valid_x)//batch_size

    if len(train_x) % batch_size != 0:
        train_steps += 1
    if len(valid_x) % batch_size != 0:
        valid_setps += 1

    """ Model """
    model_main = build_unet_main((H, W, 3))
   #print(model_main.get_weights())
    model_main.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coef, iou, Recall(), Precision()])
    # model.summary()

    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=1e-6, verbose=1),
        CSVLogger(csv_path),
        TensorBoard(),
        EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=False)
    ]

    model_main.fit(
        train_dataset,
        epochs=num_epochs,
        validation_data=valid_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_setps,
        callbacks=callbacks
    )

Train: 19 - 19
Valid: 5 - 5
Epoch 1/20
19/19 [==============================] - ETA: 0s - loss: 0.9625 - dice_coef: 0.0375 - iou: 0.0192 - recall: 0.4116 - precision: 0.3359
Epoch 1: val_loss improved from inf to 0.97905, saving model to /content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files/model.h5
19/19 [==============================] - 49s 2s/step - loss: 0.9625 - dice_coef: 0.0375 - iou: 0.0192 - recall: 0.4116 - precision: 0.3359 - val_loss: 0.9790 - val_dice_coef: 0.0210 - val_iou: 0.0106 - val_recall: 1.0000 - val_precision: 0.4030 - lr: 1.0000e-04
Epoch 2/20
19/19 [==============================] - ETA: 0s - loss: 0.9514 - dice_coef: 0.0486 - iou: 0.0249 - recall: 0.3148 - precision: 0.3814
Epoch 2: val_loss improved from 0.97905 to 0.97903, saving model to /content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files/model.h5
19/19 [==============================] - 10s 523ms/step - loss: 0.9514 - dice_coef: 0.0486 - iou: 0.0249 - recall: 0.3148 - precision: 0.3814 - 

In [6]:
main_model_weight=model_main.get_weights()

# Test

In [7]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import pandas as pd
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
from sklearn.metrics import classification_report


H = 512
W = 512

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    return ori_x, x

def read_mask(path):
    y = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    ori_y = y
    y = y/255.0
    y = y.astype(np.int32)
    return ori_y, y



def save_results(ori_x, ori_y, y_pred, save_image_path):
    line = np.ones((H, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)
    cv2.imwrite(save_image_path, cat_images)

if __name__ == "__main__":
    """ Save the results in this folder """
    create_dir("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/results1")

    """ Load the model """
    with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}):
        model = tf.keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files/model.h5")

    """ Load the dataset """
    test_x = sorted(glob("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/test/image/*"))
    test_y = sorted(glob("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/test/mask/*"))

    """ Make the prediction and calculate the metrics values """
    SCORE = []
    for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
        """ Extracting name """
        name = x.split("/")[-1].split(".")[0]

        """ Read the image and mask """
        ori_x, x = read_image(x)
        ori_y, y = read_mask(y)

        """ Prediction """
        y_pred = model.predict(np.expand_dims(x, axis=0))[0]
        y_pred = y_pred > 0.5
        y_pred = y_pred.astype(np.int32)
        y_pred = np.squeeze(y_pred, axis=-1)

        """ Saving the images """
        save_image_path = f"/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/results1/{name}.png"
        save_results(ori_x, ori_y, y_pred, save_image_path)

        """ Flatten the array """
        y = y.flatten()
        y_pred = y_pred.flatten()

        """ Calculate the metrics """
        acc_value = accuracy_score(y, y_pred)
        f1_value = f1_score(y, y_pred,labels=[0, 1], average="macro")
        jac_value = jaccard_score(y, y_pred,labels=[0, 1], average="macro")
        recall_value = recall_score(y, y_pred,labels=[0, 1], average="macro")
        precision_value = precision_score(y, y_pred,labels=[0, 1], average="macro")
        SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value])

    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)
    print()
    print(f"Accuracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f}")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")



 10%|█         | 1/10 [00:01<00:16,  1.80s/it]/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 20%|██        | 2/10 [00:03<00:13,  1.63s/it]/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 30%|███       | 3/10 [00:05<00:12,  1.86s/it]/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r


Accuracy: 0.99809
F1: 0.49952
Jaccard: 0.49905
Recall: 0.49905
Precision: 0.50000


# Model 1(test)

In [8]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet_1(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="UNET")
    return model
    

In [9]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision


H = 512
W = 512

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path):
    x = sorted(glob(os.path.join(path, "set2","image", "*")))
    y = sorted(glob(os.path.join(path, "set2","mask", "*")))
    return x, y

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)              ## (512, 512, 1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch_size=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(4)
    return dataset

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory to save files """
    create_dir("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files")

    """ Hyperparameters """
    batch_size = 1
    lr = 1e-4
    num_epochs = 20
    model_path = os.path.join("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files", "model1.h5")
    csv_path = os.path.join("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files", "data1.csv")

    """ Dataset """
    dataset_path = "/content/drive/MyDrive/Colab Notebooks/Dataset/new_data"
    train_path = os.path.join(dataset_path, "train")
    valid_path = os.path.join(dataset_path, "valid")

    train_x, train_y = load_data(train_path)
    train_x, train_y = shuffling(train_x, train_y)
    valid_x, valid_y = load_data(valid_path)

    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Valid: {len(valid_x)} - {len(valid_y)}")

    train_dataset = tf_dataset(train_x, train_y, batch_size=batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch_size=batch_size)

    train_steps = len(train_x)//batch_size
    valid_setps = len(valid_x)//batch_size

    if len(train_x) % batch_size != 0:
        train_steps += 1
    if len(valid_x) % batch_size != 0:
        valid_setps += 1

    """ Model """
    model1 = build_unet_1((H, W, 3))
    model1.set_weights(main_model_weight)
    model1.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coef, iou, Recall(), Precision()])
    # model.summary()

    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=1e-6, verbose=1),
        CSVLogger(csv_path),
        TensorBoard(),
        EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=False)
    ]

    model1.fit(
        train_dataset,
        epochs=num_epochs,
        validation_data=valid_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_setps,
        callbacks=callbacks
    )

Train: 19 - 19
Valid: 5 - 5
Epoch 1/20
19/19 [==============================] - ETA: 0s - loss: 0.9240 - dice_coef: 0.0760 - iou: 0.0395 - recall_1: 0.2695 - precision_1: 0.5030
Epoch 1: val_loss improved from inf to 0.95731, saving model to /content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files/model1.h5
19/19 [==============================] - 36s 2s/step - loss: 0.9240 - dice_coef: 0.0760 - iou: 0.0395 - recall_1: 0.2695 - precision_1: 0.5030 - val_loss: 0.9573 - val_dice_coef: 0.0427 - val_iou: 0.0218 - val_recall_1: 0.2557 - val_precision_1: 0.2199 - lr: 1.0000e-04
Epoch 2/20
19/19 [==============================] - ETA: 0s - loss: 0.9155 - dice_coef: 0.0845 - iou: 0.0441 - recall_1: 0.2580 - precision_1: 0.5211
Epoch 2: val_loss did not improve from 0.95731
19/19 [==============================] - 6s 305ms/step - loss: 0.9155 - dice_coef: 0.0845 - iou: 0.0441 - recall_1: 0.2580 - precision_1: 0.5211 - val_loss: 0.9604 - val_dice_coef: 0.0396 - val_iou: 0.0202 - val_recall_

In [10]:
test1_model_weight = model1.get_weights()

# Model2 (test)

In [11]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet_2(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="UNET")
    return model
    

In [12]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision


H = 512
W = 512

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path):
    x = sorted(glob(os.path.join(path, "set3","image", "*")))
    y = sorted(glob(os.path.join(path, "set3","mask", "*")))
    return x, y

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)              ## (512, 512, 1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch_size=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(4)
    return dataset

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory to save files """
    create_dir("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files")

    """ Hyperparameters """
    batch_size = 1
    lr = 1e-4
    num_epochs = 20
    model_path = os.path.join("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files", "model2.h5")
    csv_path = os.path.join("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files", "data2.csv")

    """ Dataset """
    dataset_path = "/content/drive/MyDrive/Colab Notebooks/Dataset/new_data"
    train_path = os.path.join(dataset_path, "train")
    valid_path = os.path.join(dataset_path, "valid")

    train_x, train_y = load_data(train_path)
    train_x, train_y = shuffling(train_x, train_y)
    valid_x, valid_y = load_data(valid_path)

    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Valid: {len(valid_x)} - {len(valid_y)}")

    train_dataset = tf_dataset(train_x, train_y, batch_size=batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch_size=batch_size)

    train_steps = len(train_x)//batch_size
    valid_setps = len(valid_x)//batch_size

    if len(train_x) % batch_size != 0:
        train_steps += 1
    if len(valid_x) % batch_size != 0:
        valid_setps += 1

    """ Model """
    model2 = build_unet_2((H, W, 3))
    model2.set_weights(main_model_weight)
    model2.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coef, iou, Recall(), Precision()])
    # model.summary()

    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=1e-6, verbose=1),
        CSVLogger(csv_path),
        TensorBoard(),
        EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=False)
    ]

    model1.fit(
        train_dataset,
        epochs=num_epochs,
        validation_data=valid_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_setps,
        callbacks=callbacks
    )

Train: 19 - 19
Valid: 5 - 5
Epoch 1/20
19/19 [==============================] - ETA: 0s - loss: 0.8801 - dice_coef: 0.1199 - iou: 0.0638 - recall_1: 0.2926 - precision_1: 0.4532
Epoch 1: val_loss improved from inf to 0.98012, saving model to /content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files/model2.h5
19/19 [==============================] - 36s 2s/step - loss: 0.8801 - dice_coef: 0.1199 - iou: 0.0638 - recall_1: 0.2926 - precision_1: 0.4532 - val_loss: 0.9801 - val_dice_coef: 0.0199 - val_iou: 0.0101 - val_recall_1: 0.0784 - val_precision_1: 0.1804 - lr: 1.0000e-04
Epoch 2/20
19/19 [==============================] - ETA: 0s - loss: 0.8566 - dice_coef: 0.1434 - iou: 0.0774 - recall_1: 0.3065 - precision_1: 0.5408
Epoch 2: val_loss did not improve from 0.98012
19/19 [==============================] - 6s 298ms/step - loss: 0.8566 - dice_coef: 0.1434 - iou: 0.0774 - recall_1: 0.3065 - precision_1: 0.5408 - val_loss: 0.9809 - val_dice_coef: 0.0191 - val_iou: 0.0096 - val_recall_

In [13]:
test2_model_weight = model2.get_weights()

In [20]:
addition = [x + y for (x, y) in zip(test1_model_weight, test2_model_weight)] 

In [22]:
average_weight = [x / 2 for x in addition]

In [24]:
test1_model_weight

[array([[[[ 3.0954098e-02, -9.2255995e-03, -3.4987129e-02, ...,
           -3.3051439e-02,  2.2297975e-02,  5.2513752e-02],
          [-1.0042253e-03, -6.6856228e-02, -7.1569338e-02, ...,
           -4.7434755e-02,  1.9383583e-02,  2.0073550e-03],
          [ 7.8476556e-02, -7.8659259e-02, -7.1790360e-02, ...,
           -1.5144391e-02,  6.0361896e-02, -3.6814854e-02]],
 
         [[-8.5063070e-02, -7.0944428e-02, -9.1360502e-02, ...,
            8.3952630e-03,  6.7071058e-02,  5.8788832e-02],
          [ 4.8414532e-02,  5.1397886e-02,  3.2471869e-02, ...,
           -9.6446186e-02, -3.8861714e-02, -6.3279490e-03],
          [-3.9997011e-02,  8.7760193e-03, -6.0415447e-02, ...,
           -9.0752296e-02, -5.9205133e-02, -5.0872094e-03]],
 
         [[-8.6420044e-02, -2.7205791e-02, -3.7845306e-02, ...,
           -6.6420749e-02, -5.7598606e-02,  6.3627675e-02],
          [ 7.8025140e-02,  3.6558799e-02, -8.3337396e-02, ...,
           -2.1870449e-02,  6.1490681e-02,  4.1973606e-02],
  

In [25]:
test2_model_weight

[array([[[[ 0.03056491, -0.01171627, -0.03006194, ..., -0.0358078 ,
            0.02230045,  0.05228239],
          [-0.00120457, -0.06602865, -0.07215857, ..., -0.04890643,
            0.01858804,  0.00165784],
          [ 0.07795889, -0.07785399, -0.07306112, ..., -0.01603081,
            0.0606396 , -0.03713254]],
 
         [[-0.08573409, -0.07298741, -0.08597244, ...,  0.00559448,
            0.06757348,  0.05856588],
          [ 0.04836337,  0.05247814,  0.03213587, ..., -0.09794627,
           -0.0393093 , -0.00662575],
          [-0.04034571,  0.00969859, -0.06158172, ..., -0.0920096 ,
           -0.05869279, -0.00531154]],
 
         [[-0.08780827, -0.02952564, -0.03283261, ..., -0.06884402,
           -0.05624129,  0.0634426 ],
          [ 0.0777564 ,  0.03745302, -0.08333687, ..., -0.02270784,
            0.06163303,  0.04168446],
          [-0.0884027 ,  0.04623258, -0.07427448, ...,  0.04611193,
           -0.01086511, -0.03434056]]],
 
 
        [[[ 0.0573546 , -0.0727700

In [26]:
average_weight

[array([[[[ 0.0307595 , -0.01047094, -0.03252453, ..., -0.03442962,
            0.02229921,  0.05239807],
          [-0.0011044 , -0.06644244, -0.07186395, ..., -0.04817059,
            0.01898581,  0.0018326 ],
          [ 0.07821772, -0.07825662, -0.07242574, ..., -0.0155876 ,
            0.06050075, -0.0369737 ]],
 
         [[-0.08539858, -0.07196592, -0.08866647, ...,  0.00699487,
            0.06732227,  0.05867736],
          [ 0.04838895,  0.05193801,  0.03230387, ..., -0.09719623,
           -0.03908551, -0.00647685],
          [-0.04017136,  0.00923731, -0.06099859, ..., -0.09138095,
           -0.05894896, -0.00519938]],
 
         [[-0.08711416, -0.02836572, -0.03533896, ..., -0.06763238,
           -0.05691995,  0.06353514],
          [ 0.07789077,  0.03700591, -0.08333713, ..., -0.02228915,
            0.06156186,  0.04182903],
          [-0.0882037 ,  0.04584343, -0.07370999, ...,  0.04644323,
           -0.01128373, -0.03423701]]],
 
 
        [[[ 0.05760477, -0.0717451

# Repeat with average weight in main model

In [28]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet_main(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="UNET")
    return model

if __name__ == "__main__":
  input_shape = (512, 512, 3)
  model_main = build_unet_main(input_shape)
  

# Train

In [29]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision


H = 512
W = 512

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path):
    x = sorted(glob(os.path.join(path, "set1","image", "*")))
    y = sorted(glob(os.path.join(path, "set1","mask", "*")))
    return x, y

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)              ## (512, 512, 1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch_size=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(4)
    return dataset

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory to save files """
    create_dir("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files")

    """ Hyperparameters """
    batch_size = 1
    lr = 1e-4
    num_epochs = 20
    model_path = os.path.join("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files", "model3.h5")
    csv_path = os.path.join("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files", "data3.csv")

    """ Dataset """
    dataset_path = "/content/drive/MyDrive/Colab Notebooks/Dataset/new_data"
    train_path = os.path.join(dataset_path, "train")
    valid_path = os.path.join(dataset_path, "valid")

    train_x, train_y = load_data(train_path)
    train_x, train_y = shuffling(train_x, train_y)
    valid_x, valid_y = load_data(valid_path)

    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Valid: {len(valid_x)} - {len(valid_y)}")

    train_dataset = tf_dataset(train_x, train_y, batch_size=batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch_size=batch_size)

    train_steps = len(train_x)//batch_size
    valid_setps = len(valid_x)//batch_size

    if len(train_x) % batch_size != 0:
        train_steps += 1
    if len(valid_x) % batch_size != 0:
        valid_setps += 1

    """ Model """
    model_main = build_unet_main((H, W, 3))
    model_main.set_weights(average_weight)
    model_main.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coef, iou, Recall(), Precision()])
    # model.summary()

    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=1e-6, verbose=1),
        CSVLogger(csv_path),
        TensorBoard(),
        EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=False)
    ]

    model_main.fit(
        train_dataset,
        epochs=num_epochs,
        validation_data=valid_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_setps,
        callbacks=callbacks
    )

Train: 19 - 19
Valid: 5 - 5
Epoch 1/20
19/19 [==============================] - ETA: 0s - loss: 0.8896 - dice_coef: 0.1104 - iou: 0.0585 - recall_3: 0.0945 - precision_3: 0.5996
Epoch 1: val_loss improved from inf to 0.94349, saving model to /content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files/model3.h5
19/19 [==============================] - 11s 411ms/step - loss: 0.8896 - dice_coef: 0.1104 - iou: 0.0585 - recall_3: 0.0945 - precision_3: 0.5996 - val_loss: 0.9435 - val_dice_coef: 0.0565 - val_iou: 0.0292 - val_recall_3: 0.0707 - val_precision_3: 0.8008 - lr: 1.0000e-04
Epoch 2/20
19/19 [==============================] - ETA: 0s - loss: 0.8796 - dice_coef: 0.1204 - iou: 0.0641 - recall_3: 0.0957 - precision_3: 0.6183
Epoch 2: val_loss improved from 0.94349 to 0.92756, saving model to /content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files/model3.h5
19/19 [==============================] - 7s 357ms/step - loss: 0.8796 - dice_coef: 0.1204 - iou: 0.0641 - recall_3: 0.0957 

In [ ]:
#main_model_weight=model_main.get_weights()

# Test

In [30]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import pandas as pd
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
from sklearn.metrics import classification_report


H = 512
W = 512

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    return ori_x, x

def read_mask(path):
    y = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    ori_y = y
    y = y/255.0
    y = y.astype(np.int32)
    return ori_y, y



def save_results(ori_x, ori_y, y_pred, save_image_path):
    line = np.ones((H, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)
    cv2.imwrite(save_image_path, cat_images)

if __name__ == "__main__":
    """ Save the results in this folder """
    create_dir("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/results2")

    """ Load the model """
    with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}):
        model = tf.keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/files/model3.h5")

    """ Load the dataset """
    test_x = sorted(glob("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/test/image/*"))
    test_y = sorted(glob("/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/test/mask/*"))

    """ Make the prediction and calculate the metrics values """
    SCORE = []
    for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
        """ Extracting name """
        name = x.split("/")[-1].split(".")[0]

        """ Read the image and mask """
        ori_x, x = read_image(x)
        ori_y, y = read_mask(y)

        """ Prediction """
        y_pred = model.predict(np.expand_dims(x, axis=0))[0]
        y_pred = y_pred > 0.5
        y_pred = y_pred.astype(np.int32)
        y_pred = np.squeeze(y_pred, axis=-1)

        """ Saving the images """
        save_image_path = f"/content/drive/MyDrive/Colab Notebooks/Dataset/new_data/results2/{name}.png"
        save_results(ori_x, ori_y, y_pred, save_image_path)

        """ Flatten the array """
        y = y.flatten()
        y_pred = y_pred.flatten()

        """ Calculate the metrics """
        acc_value = accuracy_score(y, y_pred)
        f1_value = f1_score(y, y_pred,labels=[0, 1], average="macro")
        jac_value = jaccard_score(y, y_pred,labels=[0, 1], average="macro")
        recall_value = recall_score(y, y_pred,labels=[0, 1], average="macro")
        precision_value = precision_score(y, y_pred,labels=[0, 1], average="macro")
        SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value])

    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)
    print()
    print(f"Accuracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f}")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")



 10%|█         | 1/10 [00:00<00:06,  1.29it/s]/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 20%|██        | 2/10 [00:01<00:04,  1.75it/s]/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 30%|███       | 3/10 [00:01<00:03,  2.03it/s]/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r


Accuracy: 0.97420
F1: 0.49353
Jaccard: 0.48714
Recall: 0.53710
Precision: 0.50004
